# Projeto 2 - Ciência dos Dados - Estudo sobre Ifood

Nome: Lucas Hix

Nome: Andrea Mindlin Tessler

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
# Importações

import tweepy
import math
import os.path
import pandas as pd
import json
import re
import string
from random import shuffle

___
## Autenticando no  Twitter

* Conta - Lucas Hix: ***@Peng1104Oficial***
* Conta - Andrea Tessler: ***@Andrea25519779***

In [2]:
# Dados de autenticação do twitter:

def mkAPI(filename):
    # leitura do arquivo no formato JSON
    
    with open(filename) as file:
        data = json.load(file)
    
    # Configurando a biblioteca. Não modificar
    
    auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
    auth.set_access_token(data['access_token'], data['access_token_secret'])
    
    # Cria um objeto de API
    return tweepy.API(auth)

# Conta @Peng1104Oficial
api_lucas = mkAPI("lucas-auth.pass")

# Conta @Andrea25519779
api_andrea = mkAPI("andrea-auth.pass")

apis = [api_lucas, api_andrea]

___
## Etapas do projeto:

### Funções essenciais para o projeto

In [15]:
# Função para a captura de Tweets
def getTweets(palavra_de_procura, quantidade_de_procuras_por_api):
    # Lista do resultado final da captura
    resultado = []
    
    # Para cada API gerada
    for api in apis:
        # Contagem de captura para a API
        contagem = 0
        
        # Captura de um novo tweet
        for tweet in tweepy.Cursor(api.search, q=palavra_de_procura, lang="pt", tweet_mode="extended").items():
            # Se a contagem para esta api for igual a quatidade pedida para as capturas
            if contagem == quantidade_de_procuras_por_api:
                break
            else:
                # Texto do tweet capturado
                texto = tweet.full_text.lower()
                
                #Verfica se a palavra de procura se encontra no texto:
                if palavra_de_procura.lower() in texto:
                    # Não adiciona tweet duplicado
                    if texto not in resultado:
                        resultado.append(texto)
                
                #Adiciona 1 captura a contagem de capturas para esta API
                contagem = contagem + 1
                print("{0:.0f}% das capturas realizadas".format((contagem/quantidade_de_procuras_por_api)*100), end="\r")
    
    # Embaralhando as mensagens para reduzir um possível viés
    shuffle(resultado)
    
    return resultado

# Função para limpar um texto de um Tweet
def limpar_texto(texto, marca):
    #Remove Menções
    texto = re.sub("@[^ ]*", "", texto)
    #Remove links
    texto = re.sub("http(|s):\/\/[^ ]*", "", texto)
    #Remove Pontuações
    texto = re.sub("[" + string.punctuation + "]+", "", texto)
    #Remove a marca analisada
    return re.sub("{0}".format(marca), " ", texto)

# Função para processar um valor atribuido aos tweets analisados na tabela processada 
def calcular_valor(tabela_processada, valor, marca):
    # Separa os dados da tabela com o valor selecionado
    valores = tabela_processada[tabela_processada["Valor Atribuido"] == valor]
    
    # Junta todos os tweets separados em uma string
    mensagens = limpar_texto(valores["Dados Coletados"].str.cat(), marca)
    
    # Cria uma series contendo todas as palavras das mensagens
    series = pd.Series(mensagens.split())
    
    # Cria uma series contendo a probabilidade de cada palavra estar na categoria selecionada
    return series.value_counts(True)

### Escolha de uma marca e coleta das mensagens

Separação da marca a ser analisada e a quantiade de capturas a ser feitas para cada api

In [4]:
# Escolha da marca a ser analisada
marca = "Ifood"

# Esolha de quantidade por api (no caso quantidade*2)
quantidade_por_api = 500

Preparação para a nomeação dos Excels

In [6]:
# Numero de capturas ja realizadas
numero_de_capturas_realizadas = 1

# Verifica dentro do diretorio o numero de capturas realizadas
for file in os.listdir("."):
    if re.search("{0} \d+".format(marca), file):
        numero_de_capturas_realizadas = numero_de_capturas_realizadas + 1

# Cria o nome do excel a ser criado
nome_base_do_arquivo = marca + "-" + str(numero_de_capturas_realizadas) 

nome_do_arquivo = nome_base_do_arquivo + ".xlsx"

Captura de dados

In [10]:
dados = getTweets(marca, quantidade_por_api)

Salva os dados capturados em um Execel

In [12]:
tabela = pd.DataFrame({"Dados Coletados": pd.Series(dados), "Valor Atribuido": [""]*len(dados)})
tabela.to_excel(nome_do_arquivo, index = False)

print("O Excel", nome_do_arquivo, "foi gerado!")

O Excel Ifood-1.xlsx foi gerado!


___
### Classificando as mensagens na coragem

Esta etapa é manual, feita pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [17]:
# Transforma o excel avaliado em um DataFrame
tabela_processada = pd.read_excel("ifood classificado.xlsx")

In [19]:
# Pega os valores atribuidos na tabela para cada tweet e cria as probabilidade da palavra na categoria
contra = calcular_valor(tabela_processada, 1, marca)
favoravel = calcular_valor(tabela_processada, 0, marca)

favoravel.head(5)

que    0.030383
de     0.027741
o      0.022457
e      0.019815
não    0.018494
dtype: float64

In [20]:
# Cria um dicionario para o naive-bayes
dict_de_atribuicoes = {0 : favoravel, 1: contra}

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [26]:
# Função para executar um teste dando dados e um dicionanrio contendo as atribuições
def executar_teste(dados_de_teste, dict_de_atribuicoes, marca):
    # Lista contendo os resultados para cada dado de teste
    resultado = []
    
    for mensagem in dados_de_teste:
        # Aramazena as probabilidades de cada atribuição
        probabilidades = {}
        
        # Separa as palavras na mensagem
        for palavra in limpar_texto(mensagem, marca).split():
            # Para cada tipo de atribuição
            for tipo, probabilidade in dict_de_atribuicoes.items():
                # Verifica se a palavra se encontra em uma das atribuições
                if palavra in probabilidade:
                    # Adiciona a probabilidade da palavra ao conjunto das probabilidades
                    if tipo in probabilidades:
                        probabilidades[tipo] = probabilidades[tipo] + probabilidade[palavra]
                    else:
                        probabilidades[tipo] = probabilidade[palavra]
        
        # Adiciona ao resultado o valor atribuido a esta mensagem
        
        # Não corespondeu a nenhuma probabilidade
        if len(probabilidades) == 0:
            resultado.append(-1)
        else:
            resultado.append(max(probabilidades, key=probabilidades.get))
    
    return resultado

In [24]:
# Nova captura de dados
teste = getTweets(marca, quantidade_por_api)

In [27]:
# Gera o resultado do teste
resultado_do_teste = executar_teste(teste, dict_de_atribuicoes, marca)

In [29]:
# Salva o teste em um arquivo excel
nome_do_arquivo_de_teste = nome_base_do_arquivo + "-Teste.xlsx"

tabela = pd.DataFrame({"Dados Coletados": pd.Series(teste), "Valor Atribuido Automatico": resultado_do_teste,
                       "Valor Atribuido": [""]*len(teste)})

tabela.to_excel(nome_do_arquivo_de_teste, index = False)

print("O Excel", nome_do_arquivo_de_teste, "foi gerado!")

O Excel Ifood-1-Teste.xlsx foi gerado!


___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**